In [1]:
import os 
os.chdir('..')

import pandas as pd
import tellurium as te
from src.odbm.odbm_main import ModelBuilder


In [9]:
#Define DataFrames of species, reactions from input model
model_rxns = pd.read_csv('src/frenda_brenda/Files/Reduced Complexity/Reaction_VN_Metabolism_Gly_BIOSYN.csv')
model_species = pd.read_csv('src/frenda_brenda/Files/Reduced Complexity/SpeciesBaseMechanisms_VN_Metabolism_Gly_BIOSYN.csv')

# model_rxns['Id'] = model_rxns['Enzyme']+'; ' +model_rxns['Substrates']+'; '+model_rxns['Products']
# model_rxns.drop_duplicates(subset='Id', inplace=True)

In [10]:
#Generate and save model to txt file
modelfile = 'src/frenda_brenda/Files/Reduced Complexity/240606_VN_Metabolism_Gly_BIOSYN.txt'
myModel = ModelBuilder(model_species, model_rxns)
myModel.saveModel(modelfile)

r = []
while not r:
    try:
        r  = te.loada(myModel.compile())
    except Exception as e:
        myModel.addSpecies(str(e).split("'")[1], 0.001)
        print(str(e).split("'")[1])

# s = r.simulate(0,10)

In [8]:
with open('models/240606_VN_Metabolism_Gly_BIOSYN.sbml', 'w') as f:
    # Write the data to the file
    f.write(r.getSBML())